# Notebook to prepare the auxiliary data for the Casa Grande site for the History project

The following data are prepared:
- reference lidar 1 m DEM over Maricopa country, provided by the USGS: https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/1m/Projects/AZ_MaricopaPinal_2020_B20/
- Copernicus 30 m DEM: 1x1 degree tiles are downloaded, merged and reprojected on the same horizontal and vertical reference system as lidar DEM. It is then coregistered horizontally with a Nuth & Kaan (2011) algorithm and vertically by calculating a median shift in stable areas. Stable areas are defined as bare land, grassland and shrubland in the ESA worldcover (see below) and excluding sibsiding land (see below).
- ESA worldcover dataset 10 m: each 1x1 degree tile are downloaded from the S3 bucket and merged
- Land subsidence vector file downloaded from https://azgeo-open-data-agic.hub.arcgis.com/datasets/azwater::land-subsidence/explore


In [ ]:
import os
import subprocess
from glob import glob
import folium

import xdem
import matplotlib.pyplot as plt
import geoutils as gu
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
import rasterio

# import local functions for the downloading
sys.path.append(os.path.abspath("../src/"))
import history.aux_data.download_tools as dt

## Global settings

In [16]:
VISUALIZATION = True
OVERWRITE = False

# PATH SETTINGS

OUTPUT_DIRECTORY = "./data/casa_grande/aux_data/"

# final generated files
LARGE_DEM_FILE = os.path.join(OUTPUT_DIRECTORY, "reference_dem_large.tif")
ZOOM_DEM_FILE = os.path.join(OUTPUT_DIRECTORY, "reference_dem_zoom.tif")
LARGE_DEM_MASK_FILE = os.path.join(OUTPUT_DIRECTORY, "reference_dem_large_mask.tif")
ZOOM_DEM_MASK_FILE = os.path.join(OUTPUT_DIRECTORY, "reference_dem_zoom_mask.tif")

# temporary directory to download tiles and process
TMP_DIRECTORY = os.path.join(OUTPUT_DIRECTORY, "tmp")

LANDCOVER_DIRECTORY = os.path.join(TMP_DIRECTORY, "landcover")
LIDARDEM_DIRECTORY = os.path.join(TMP_DIRECTORY, "lidar_dem")
COP30DEM_DIRECTORY = os.path.join(TMP_DIRECTORY, "cop30_dem")

os.makedirs(TMP_DIRECTORY, exist_ok=True)

## ! IMPORTANT !

you need to download the land subsidence mask manually at: https://azgeo-open-data-agic.hub.arcgis.com/datasets/azwater::land-subsidence/explore


In [9]:
land_subsidence_file = os.path.join(TMP_DIRECTORY, "Land_Subsidence.geojson")
assert os.path.exists(land_subsidence_file)

## Define bounding box

We define 2 bounding box : 
- `zoom_bounds` : for lidar DEM of 1m resolution
- `large_bounds` : for copernicus DEM of 30m resolution

In [11]:
# Bounding box of the two area of interest
zoom_bounds = [414000, 3613020, 444000, 3650010]
large_bounds = [261990, 3405030, 612990, 3880020]
epsg_str = "EPSG:26912"

# Save to gjson files
zoom_gdf = gpd.GeoDataFrame(geometry=[gu.projtools.bounds2poly(zoom_bounds),], crs=epsg_str)
zoom_gdf.to_file(os.path.join(TMP_DIRECTORY, "zoom_aoi.geojson"))

large_gdf = gpd.GeoDataFrame(geometry=[gu.projtools.bounds2poly(large_bounds),], crs=epsg_str)
large_gdf.to_file(os.path.join(TMP_DIRECTORY, "large_aoi.geojson"))

zoom_bounds_latlon = gu.projtools.bounds2poly(zoom_bounds, in_crs=epsg_str, out_crs="EPSG:4326").bounds
large_bounds_latlon = gu.projtools.bounds2poly(large_bounds, in_crs=epsg_str, out_crs="EPSG:4326").bounds

# and swap axes for lon/lat
zoom_bounds_lonlat = [zoom_bounds_latlon[k] for k in [1, 0, 3, 2]]
large_bounds_lonlat = [large_bounds_latlon[k] for k in [1, 0, 3, 2]]

## [Optional] Visualisation of bounding box and land subsidence

In [17]:
if VISUALIZATION:
    gdf = gpd.read_file(land_subsidence_file)

    centroid_proj = gdf.to_crs("EPSG:26912").union_all().centroid
    centroid_ll = gpd.GeoSeries([centroid_proj], crs="EPSG:26912").to_crs("EPSG:4326").geometry[0]

    m = folium.Map(location=[centroid_ll.y, centroid_ll.x], zoom_start=7)
    folium.GeoJson(data=gdf).add_to(m)
    folium.GeoJson(data=zoom_gdf.to_crs(epsg=4326)).add_to(m)
    folium.GeoJson(data=large_gdf.to_crs(epsg=4326)).add_to(m)
m